In [67]:
# CSV 파일 전처리
# 엑셀로 만든 CSV 파일 인코딩을 엑셀에서 UTF-8로 저장해도 UTF-8 이나 EUC-KR 로 인코딩할 때 에러 발생
# CP949로 인코딩 성공

def clean_file(fileName, save=False, replace=False):
    import pandas as pd
    if replace:
        save = True
    [fname, ftype] = str(fileName).split('.')
    try:
        if ftype == 'csv':
            df = pd.read_csv(fileName)
            print(fileName+ ' is loaded with encoding = UTF-8')
        elif ftype == 'xlsx':
            df = pd.read_excel(fileName)
        df = col_check(df)

        if not replace:
            fileName = fname+'_cleaned.'+ftype

        if save:
            if ftype == 'csv':
                df.to_csv(fileName, index=False)

            elif ftype == 'xlsx':
                df.to_excel(fileName, index=False)
        return df

    except UnicodeDecodeError:
        df = pd.read_csv(fileName, encoding='CP949')
        print(fileName+ ' is loaded with encoding = CP949')
        df = col_check(df)
        return df

    except Exception as ex:
        print('%s' % (ex))


def col_check(df):
    colList = list(df.columns)
    for col in colList:
        if "Unnamed:" in col:  # 이름이 없는 칼럼 삭제
            df = df.drop(col, axis=1)
            colList = list(df.columns)
    newcolList = []
    for col in colList:
        if "\t" in col:
            new = str(col).replace('\t', '')
        else:
            new = col
        newcolList.append(new)
    df.columns = newcolList
    return df



In [68]:
fDir = 'C:/vscodeFiles/'
fileName = 'result.csv'
df = clean_file(fDir+fileName)
df['거래처코드'] = df['거래처코드'].astype({'거래처코드':'str'})
df['거래처코드'].str.replace('.0', '')
df.to_csv(fDir+'2'+fileName, encoding='UTF-8')

C:\Users\se2p0\AppData\Local\Temp\ipykernel_12796\1210212321.py:12: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fileName)


In [52]:
df = clean_file(fDir+'2'+fileName)


C:\Users\se2p0\AppData\Local\Temp\ipykernel_12796\2593312506.py:12: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fileName)


bool

In [63]:
for i in range(len(df['합계'])):
    if type(df['합계'][i]) != int:
        df['합계'][i]

In [65]:
df.

구분        False
일자        False
품목구분       True
품목코드      False
품목명       False
최신 품목명    False
규격         True
수량        False
단위         True
단가        False
공급가       False
합계        False
거래처코드      True
거래처        True
거래처그룹1     True
거래처그룹2     True
담당자        True
개설연도       True
지역구분       True
인센가입       True
품목그룹1      True
품목그룹2      True
인센품목       True
dtype: bool

In [ ]:
sales_all = pd.read_sql_table('sales_all', con=engine)
sales_all


In [ ]:
# 해당 모듈들이 설치되어 있어야 함
# sqlalchemy, pymssql
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import select
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text
from sqlalchemy.engine.url import URL
from sqlalchemy.types import NVARCHAR

db_url = {
    'database': "dbo.sales_main",
    'drivername': 'mssql+pymssql',
    'username': 'se2p00',
    'password': 'rmeo9821',
    'host': 'fz_sa_dgkim',
    'port': '1433',
    'query': {'charset': 'UTF-8'},  # the key-point setting
}

engine = create_engine(URL(**db_url), echo=True)

# engine = create_engine('mssql+pymssql://se2p00:rmeo9821@FZ_SA_DGKIM/dbo.sales_main', echo=True)
# session 생성
sales_all = pd.read_sql_table('sales_all', con=engine)
sales_all

In [ ]:
# Table 생성 테스트

from sqlalchemy.types import NVARCHAR, DATE, Integer
dataTypeList = [NVARCHAR]*(len(df.columns)-1)
columnList = list(df.columns)
dTypeDict = dict(zip(columnList, dataTypeList))
dTypeDict['일자'] = DATE
dTypeDict['단가'] = Integer
dTypeDict['공급가'] = Integer
dTypeDict['합계'] = Integer
dTypeDict['수량'] = Integer

df = sales_all
conn = engine.connect()
df.to_sql(name='sales_sample', con=engine,
          if_exists='replace', index=False, dtype=dTypeDict)
conn.close()
sales_sample = pd.read_sql_table('sales_sample', con=engine)


In [ ]:
# 조건에 해당하는 데이터 뽑기
newdf = sales_sample[sales_sample['단가'] > 1000000]
newdf.to_excel('sample.xlsx')


In [ ]:
import pymssql

# DB 서버 주소
server = 'FZ_SA_DGKIM'
# 데이터 베이스 이름
database = 'dbo.sales_main'
# 접속 유저명
username = 'se2p00'
# 접속 유저 패스워드
password = 'rmeo9821'


# MSSQL 접속
cnxn = pymssql.connect(server, username, password, database)
cursor = cnxn.cursor()

# SQL문 실행
cursor.execute('SELECT * FROM dbo.sales_all;')
